In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import os


Data Preparation

In [21]:
import pandas as pd
import os

base_path = "/Users/anhuynh/Downloads/archive (5)/"

file_month_map = {
    "November sleep data - Sheet1.csv": "November",
    "December sleep data - Sheet1.csv": "December",
    "January sleep data - Sheet1.csv": "January",
    "February sleep data - Sheet1 (1).csv": "February",
    "March sleep data - Sheet1.csv": "March",
    "April sleep data - Sheet1.csv": "April"
}

monthly_dfs = []

# Process each file
for filename, month in file_month_map.items():
    file_path = os.path.join(base_path, filename)
    df = pd.read_csv(file_path)

    # Rename inconsistent columns
    df.rename(columns={df.columns[0]: "Weekday"}, inplace=True)
    df.rename(columns={
        "SLEEP SQORE": "SLEEP SCORE",
        "HEARTRATE BELOW RESTING": "HEART RATE BELOW RESTING",
        "HEART RATE UNDER RESTING": "HEART RATE BELOW RESTING"
    }, inplace=True)

    # Add month column
    df["Month"] = month

    # Convert 'DATE' to datetime and drop invalid ones
    df["Date"] = pd.to_datetime(df["DATE"], errors="coerce")
    df = df.dropna(subset=["Date"])

    # Extract actual weekday
    df["Weekday"] = df["Date"].dt.day_name()

    # Convert percentage columns to decimal
    percent_cols_to_convert = ["REM SLEEP", "DEEP SLEEP", "HEART RATE BELOW RESTING"]
    for col in percent_cols_to_convert:
        df[col] = df[col].str.replace('%', '', regex=False).astype(float) / 100

    # Split 'SLEEP TIME'
    if "SLEEP TIME" in df.columns:
        df["SLEEP START"] = df["SLEEP TIME"].str.split("-").str[0].str.strip()
        df["SLEEP END"] = df["SLEEP TIME"].str.split("-").str[1].str.strip()

    # Convert 'SLEEP START' and 'SLEEP END' to hour
    df['SLEEP START'] = pd.to_datetime(df['SLEEP START'], errors='coerce').dt.hour
    df['SLEEP END'] = pd.to_datetime(df['SLEEP END'], errors='coerce').dt.hour

    # Keep only expected columns
    expected_cols = [
        "Month", "Weekday", "Date", "SLEEP SCORE", "HOURS OF SLEEP", "REM SLEEP",
        "DEEP SLEEP", "HEART RATE BELOW RESTING", "SLEEP TIME", "SLEEP START", "SLEEP END"
    ]
    df = df[[col for col in expected_cols if col in df.columns]]

    # Append the clean df
    monthly_dfs.append(df)

# Combine all months
df = pd.concat(monthly_dfs, ignore_index=True)



/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/468405663.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['SLEEP START'] = pd.to_datetime(df['SLEEP START'], errors='coerce').dt.hour
/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/468405663.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['SLEEP END'] = pd.to_datetime(df['SLEEP END'], errors='coerce').dt.hour
/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/468405663.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['SLEEP START'] = pd.to_datetime(df['SLE

In [22]:
# Function to convert HH:MM or HH:MM:SS to total seconds
def time_to_seconds(t):
    try:
        parts = t.split(":")
        parts = list(map(int, parts))
        if len(parts) == 2:  # HH:MM
            h, m = parts
            s = 0
        elif len(parts) == 3:  # HH:MM:SS
            h, m, s = parts
        else:
            return np.nan
        return h * 3600 + m * 60 + s
    except:
        return np.nan

#print(df['Month'].unique())

# Apply to your column
df["SLEEP DURATION SECONDS"] = df["HOURS OF SLEEP"].apply(time_to_seconds)

# Also useful to get decimal hours
df["SLEEP DURATION HOURS"] = df["SLEEP DURATION SECONDS"] / 3600

### Combine all of the months into a single DataFrame
combined_data = pd.concat(monthly_dfs, ignore_index=True)

### Data cleaning
data = combined_data.dropna()

# Display the cleaned data
print(data.head())

      Month    Weekday       Date  SLEEP SCORE HOURS OF SLEEP  REM SLEEP  \
0  November     Monday 2021-11-01         88.0        8:06:00       0.20   
1  November    Tuesday 2021-11-02         83.0        7:57:00       0.12   
2  November  Wednesday 2021-11-03         81.0        7:06:00       0.13   
3  November   Thursday 2021-11-04         86.0        7:04:00       0.19   
4  November     Friday 2021-11-05         81.0        9:24:00       0.17   

   DEEP SLEEP  HEART RATE BELOW RESTING        SLEEP TIME  SLEEP START  \
0        0.13                      0.84  10:41pm - 7:54am         22.0   
1        0.18                      0.90  10:40pm - 7:55am         22.0   
2        0.22                      0.93  11:03pm - 7:16am         23.0   
3        0.17                      0.97  10:55pm - 6:56am         22.0   
4        0.15                      0.66  10:14pm - 9:01am         22.0   

   SLEEP END  
0        7.0  
1        7.0  
2        7.0  
3        6.0  
4        9.0  


In [23]:

# Function to convert HH:MM or HH:MM:SS to total seconds
def time_to_seconds(t):
    try:
        parts = t.split(":")
        parts = list(map(int, parts))
        if len(parts) == 2:  # HH:MM
            h, m = parts
            s = 0
        elif len(parts) == 3:  # HH:MM:SS
            h, m, s = parts
        else:
            return np.nan
        return h * 3600 + m * 60 + s
    except:
        return np.nan

# Apply to your column
data["SLEEP DURATION SECONDS"] = data["HOURS OF SLEEP"].apply(time_to_seconds)

# Also useful to get decimal hours
data["SLEEP DURATION HOURS"] = data["SLEEP DURATION SECONDS"] / 3600


/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/3228070899.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SLEEP DURATION SECONDS"] = data["HOURS OF SLEEP"].apply(time_to_seconds)
/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/3228070899.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["SLEEP DURATION HOURS"] = data["SLEEP DURATION SECONDS"] / 3600


In [24]:
###more feature engineering 

#prev sleep diff
data['PREV_SLEEP_START'] = data['SLEEP START'].shift(1)
data['PREV_SLEEP_START'] = data['SLEEP START'].shift(1).fillna(0)

# sleep start var
data['SLEEP_START_VARIABILITY'] = (data['SLEEP START'] - data['PREV_SLEEP_START']).abs()
data['SLEEP_START_VARIABILITY']  = data['SLEEP_START_VARIABILITY'] .fillna(0)

#weekend or not
data['Is_Weekend'] = data['Weekday'].isin(['Saturday', 'Sunday']).astype(int)

#seasons
season_map = {
    'December': 'Winter', 'January': 'Winter', 'February': 'Winter',
    'March': 'Spring', 'April': 'Spring', 'May': 'Spring',
    'June': 'Summer', 'July': 'Summer', 'August': 'Summer',
    'September': 'Fall', 'October': 'Fall', 'November': 'Fall'
}

data['Season'] = data['Month'].map(season_map)


data

/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/318578420.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['PREV_SLEEP_START'] = data['SLEEP START'].shift(1)
/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/318578420.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['PREV_SLEEP_START'] = data['SLEEP START'].shift(1).fillna(0)
/var/folders/v9/hcdjkhvd5nq6llrn61tfl3qr0000gn/T/ipykernel_48638/318578420.py:8: SettingWithCopyWarning: 
A value is trying to be set on

,Month,Weekday,Date,SLEEP SCORE,HOURS OF SLEEP,REM SLEEP,DEEP SLEEP,HEART RATE BELOW RESTING,SLEEP TIME,SLEEP START,SLEEP END,SLEEP DURATION SECONDS,SLEEP DURATION HOURS,PREV_SLEEP_START,SLEEP_START_VARIABILITY,Is_Weekend,Season
0,November,Monday,2021-11-01,88.0,8:06:00,0.20,0.13,0.84,10:41pm - 7:54am,22.0,7.0,29160,8.100000,0.0,22.0,0,Fall
1,November,Tuesday,2021-11-02,83.0,7:57:00,0.12,0.18,0.90,10:40pm - 7:55am,22.0,7.0,28620,7.950000,22.0,0.0,0,Fall
2,November,Wednesday,2021-11-03,81.0,7:06:00,0.13,0.22,0.93,11:03pm - 7:16am,23.0,7.0,25560,7.100000,22.0,1.0,0,Fall
3,November,Thursday,2021-11-04,86.0,7:04:00,0.19,0.17,0.97,10:55pm - 6:56am,22.0,6.0,25440,7.066667,23.0,1.0,0,Fall
4,November,Friday,2021-11-05,81.0,9:24:00,0.17,0.15,0.66,10:14pm - 9:01am,22.0,9.0,33840,9.400000,22.0,0.0,0,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,April,Tuesday,2022-04-26,85.0,7:18:00,0.22,0.14,1.00,9:32pm - 6:00am,21.0,6.0,26280,7.300000,21.0,0.0,0,Spring
177,April,Wednesday,2022-04-27,90.0,7:34:00,0.24,0.19,0.98,9:19pm - 5:49am,21.0,5.0,27240,7.566667,21.0,0.0,0,Spring
178,April,Thursday,2022-04-28,87.0,6:54:00,0.21,0.22,0.90,10:02pm - 5:46am,22.0,5.0,24840,6.900000,21.0,1.0,0,Spring
179,April,Friday,2022-04-29,86.0,7:45:00,0.19,0.17,0.95,10:15pm - 7:24am,22.0,7.0,27900,7.750000,22.0,0.0,0,Spring


In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

#final data Prep 


# Define the columns
numeric_cols = ['SLEEP SCORE', 'REM SLEEP', 'DEEP SLEEP', 'HEART RATE BELOW RESTING', 'SLEEP START', 'SLEEP END', 'SLEEP DURATION SECONDS', 'SLEEP DURATION HOURS', 'SLEEP_START_VARIABILITY', 'PREV_SLEEP_START','Is_Weekend' ]
categorical_cols = ['Month', 'Weekday', 'Season']

# Define transformers for numeric and categorical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Handle missing values
    ('scaler', StandardScaler())  # Standardize numeric variables
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encoding
])

# Combine transformations in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Apply the transformations to the DataFrame
df_transformed = preprocessor.fit_transform(data)

# Convert the result back to a DataFrame with appropriate column names
# For one-hot encoded columns, the names are generated dynamically
categorical_feature_names = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_cols)
column_names = numeric_cols + list(categorical_feature_names)

df_standardized_encoded = pd.DataFrame(df_transformed, columns=column_names)



Model Building

In [26]:
# Select features and target
target = "SLEEP SCORE"
X = df_standardized_encoded.drop(columns=[target])
y = df_standardized_encoded[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)


Bagging Models

In [ ]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import numpy as np

# Create the advanced bagging model using RandomForestRegressor as the base estimator
bagging_model = BaggingRegressor(
    estimator=RandomForestRegressor(n_estimators=100, random_state=1),
    n_estimators=100,  # Number of bagging iterations
    random_state=1
)

# Perform cross-validation on Bagging model for MSE
bagging_cv_mse_scores = cross_val_score(
    bagging_model, X, y, cv=3, scoring='neg_mean_squared_error'
)
bagging_cv_mse = -np.mean(bagging_cv_mse_scores)  # Convert negative MSE to positive
bagging_cv_mse_std = np.std(bagging_cv_mse_scores)

# Perform cross-validation on Bagging model for R²
bagging_cv_r2_scores = cross_val_score(
    bagging_model, X, y, cv=5, scoring='r2'
)
bagging_cv_r2 = np.mean(bagging_cv_r2_scores)
bagging_cv_r2_std = np.std(bagging_cv_r2_scores)

# Output the cross-validation results
print(f"Bagging CV MSE: {bagging_cv_mse:.2f}")
print(f"CV Standard Deviation (MSE): {bagging_cv_mse_std:.2f}")
print(f"Bagging CV R²: {bagging_cv_r2:.3f}")
print(f"CV Standard Deviation (R²): {bagging_cv_r2_std:.3f}")


Bagging CV MSE: 0.68
CV Standard Deviation (MSE): 0.96
Bagging CV R²: 0.492
CV Standard Deviation (R²): 0.248


In [33]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import numpy as np

# Create the advanced bagging model using DecisionTreeRegressor as the base estimator
bagging_model = BaggingRegressor(
    estimator=DecisionTreeRegressor(random_state=1),
    n_estimators=100,  # Number of bagging iterations
    random_state=1
)

# Perform cross-validation on Bagging model for MSE
bagging_cv_mse_scores = cross_val_score(
    bagging_model, X, y, cv=3, scoring='neg_mean_squared_error'
)
bagging_cv_mse = -np.mean(bagging_cv_mse_scores)  # Convert negative MSE to positive
bagging_cv_mse_std = np.std(bagging_cv_mse_scores)

# Perform cross-validation on Bagging model for R²
bagging_cv_r2_scores = cross_val_score(
    bagging_model, X, y, cv=5, scoring='r2'
)
bagging_cv_r2 = np.mean(bagging_cv_r2_scores)
bagging_cv_r2_std = np.std(bagging_cv_r2_scores)

# Output the cross-validation results
print(f"Bagging CV MSE: {bagging_cv_mse:.2f}")
print(f"CV Standard Deviation (MSE): {bagging_cv_mse_std:.2f}")
print(f"Bagging CV R²: {bagging_cv_r2:.3f}")
print(f"CV Standard Deviation (R²): {bagging_cv_r2_std:.3f}")


Bagging CV MSE: 0.80
CV Standard Deviation (MSE): 0.59
Bagging CV R²: 0.306
CV Standard Deviation (R²): 0.364


Stacking Models

In [27]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Assuming X_train, X_test, y_train, and y_test are already defined

# Define base models for stacking
base_models = [
    ('lr', LinearRegression()),        # Linear regression
    ('dt', DecisionTreeRegressor()),   # Decision tree regressor
    ('rf', RandomForestRegressor())    # Random forest regressor
]

# Define the meta-model
meta_model = LinearRegression()

# Create the stacking model
stacking_model = StackingRegressor(estimators=base_models, final_estimator=meta_model)

# Train the model
stacking_model.fit(X_train, y_train)

# Predict on the test set
y_pred = stacking_model.predict(X_test)

# Evaluate the model using Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Mean Squared Error: 0.1451374368948915
R-squared: 0.785948558182254


In [28]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Base models
base_models = [
    ('ridge', Ridge(alpha=1.0)),  # Ridge regression model
    ('svr', SVR(kernel='rbf')),   # Support Vector Regression model
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42))  # Random Forest Regressor model
]

# Final estimator: simple Ridge
final_estimator = Ridge(alpha=1.0)

# Create the stacking model
stacking_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator)

# Train-test split
target = "SLEEP SCORE"
X = df_standardized_encoded.drop(columns=[target])
y = df_standardized_encoded[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Train the model
stacking_model.fit(X_train, y_train)

# Predict on the test set
y_pred = stacking_model.predict(X_test)

# Evaluate the model using Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")



Mean Squared Error: 0.13227985721255153
R-squared: 0.8049111603073319


In [29]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Base models
base_models = [
    ('ridge', Ridge()),  # Ridge regression model
    ('svr', SVR()),      # Support Vector Regression model
    ('rf', RandomForestRegressor(random_state=42))  # Random Forest Regressor model
]

# Final estimator: Ridge (initial choice)
final_estimator = Ridge()

# Create the stacking model
stacking_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator)

# Define the hyperparameter grid for base models and final estimator
param_grid = {
    'ridge__alpha': [0.1, 1.0, 10.0],
    'svr__C': [0.1, 1.0, 10.0],
    'svr__gamma': ['scale', 'auto'],
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 10, 20],
    'final_estimator__alpha': [0.1, 1.0, 10.0]
}

# Train-test split
target = "SLEEP SCORE"
X = df_standardized_encoded.drop(columns=[target])
y = df_standardized_encoded[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Perform GridSearchCV to tune hyperparameters
grid_search = GridSearchCV(estimator=stacking_model, param_grid=param_grid, cv=2, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Predict using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model using Mean Squared Error and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Best parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


Best parameters: {'final_estimator__alpha': 0.1, 'rf__max_depth': 10, 'rf__n_estimators': 100, 'ridge__alpha': 1.0, 'svr__C': 1.0, 'svr__gamma': 'auto'}
Mean Squared Error: 0.1198975598506005
R-squared: 0.823172807061237


The best-performing model among all those tested, including stacking and bagging, was the StackingRegressor combining Ridge regression, SVR, and Random Forest, which achieved an impressive R-squared of 0.823. This result makes sense, as stacking typically outperforms bagging. By blending the strengths of different models, stacking captures a wider variety of data patterns, leading to more accurate and robust predictions.

In [32]:
# Feature importances (only for models that support it like RandomForest)
features = X_train.columns
rf_imp = best_model.named_estimators_['rf'].feature_importances_

# SVR and Ridge do not have `feature_importances_`, so handle them separately
feat_imp = pd.Series(rf_imp, index=features).sort_values(ascending=False)

# Print the top 10 feature importances
print(feat_imp.head(10))

HEART RATE BELOW RESTING    0.365402
REM SLEEP                   0.209072
Weekday_Thursday            0.104761
SLEEP DURATION HOURS        0.085880
SLEEP DURATION SECONDS      0.078780
Month_March                 0.030082
DEEP SLEEP                  0.028720
SLEEP_START_VARIABILITY     0.014377
Season_Winter               0.011888
PREV_SLEEP_START            0.011504
dtype: float64


Based on the best stacking model, the strongest numerical predictors of SLEEP SCORE are:

HEART RATE BELOW RESTING (0.36) – Higher proportions of time spent below resting heart rate are strongly associated with higher sleep scores.

REM SLEEP (0.20) – More REM sleep correlates with better sleep quality, contributing significantly to higher sleep scores.

Weekday_Thursday (0.10) – Sleep scores tend to be higher on Thursdays, possibly reflecting weekly sleep pattern trends.

SLEEP DURATION HOURS (0.09) – Longer sleep duration in hours has a modest positive effect on sleep scores.

SLEEP DURATION SECONDS (0.08) – Consistent with the above, total sleep time in seconds also shows a small positive relationship with sleep scores.

DEEP SLEEP (0.03) – Slightly contributes to higher sleep scores, though weaker than REM sleep or total duration.

SLEEP_START_VARIABILITY (0.01) – Lower variability in sleep start times is weakly associated with higher sleep scores.

Season_Winter (0.01) – Slight seasonal effect; winter shows a marginal positive association with sleep scores.

PREV_SLEEP_START (0.01) – The timing of the previous sleep start has a weak positive relationship with the current sleep score.

Month_March (0.03) – Marginal seasonal effect; March shows a slight positive association with sleep scores.